In [1]:
import spotipy
import base64
import requests
from creds import CLIENT_ID, CLIENT_SECRET

# "keep it secret, keep it safe." - Gandalf the Grey, Senior Developer
credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
credentials_base64 = base64.b64encode(credentials.encode())

#Make a request for the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Access token not obtained.")
    exit()

Access token obtained successfully.
